# Supporting Material for BBI Paper - XY Table Stuff

The XYZ table was done *after* the initial work using just a vice. So this is done as another project.

## Instrument Connection Scripts

The following script will connect to various instruments in use. See the main script for details of the instruments.

### DP832 - Power Supply

To control the ChipJabber-SimpleBBI probe, an external power supply is used. The authors had a DP832 on-hand and choose to use this, the following is required to control the power supply to adjust the voltage.

In [1]:
import visa

class DP832(object):
    def __init__(self):
        pass

    def conn(self, constr='USB0::6833::3601::DP8C152800994::0::INSTR'):
        """Attempt to connect to instrument"""
        rm = visa.ResourceManager()
        self.inst = rm.open_resource(constr)

    def identify(self):
        """Return identify string which has serial number"""
        return self.inst.query("*IDN?")

    def readings(self, channel="CH1"):
        """Read voltage/current/power from CH1/CH2/CH3"""       
        resp = self.inst.query("MEAS:ALL? %s"%channel)
        resp = resp.split(',')
        dr = {"v":float(resp[0]), "i":float(resp[1]), "p":float(resp[2])}
        return dr
    
    def set_voltage(self, voltage, channel="CH1"):
        vstring = "%f"%voltage
        self.inst.write(":APPL " + channel + ", " + vstring)
        
    def set_output(self, enabled, channel="CH1"):
        if enabled:
            state = "ON"
        else:
            state = "OFF"
        self.inst.write(":OUTP " + channel + ", " + state)

    def dis(self):
        del self.inst

    def writing(self, command=""):
        self.inst.write(command)


In [3]:
psu = DP832()
psu.conn()
psu.set_output(True, "CH1")
psu.set_voltage(0.0, "CH1")

ps = None

def set_voltage(v):
    psu.set_voltage(v, "CH1")

get_capabilities() failed - some devices don't work on this, skipping


### ChipWhisperer Setup

The following connects to the ChipWhisperer, and is used before running any of the other sections.

In [180]:
PLATFORM="CW308_STM32F4"
%run "../Helper_Scripts/Setup_Generic.ipynb"
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "enable_only"
scope.glitch.trigger_src = "ext_single"
scope.glitch.resetDcms()
scope.glitch.repeat = 5
scope.glitch.ext_offset = 260

cwlite Device
fw_version = 
    major = 0
    minor = 11
    debug = 0
gain = 
    mode = high
    gain = 30
    db   = 24.8359375
adc = 
    state      = False
    basic_mode = rising_edge
    timeout    = 2
    offset     = 0
    presamples = 0
    samples    = 5000
    decimate   = 1
    trig_count = 0
clock = 
    adc_src       = clkgen_x4
    adc_phase     = 0
    adc_freq      = 29538459
    adc_rate      = 29538459.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1       = serial_rx
    tio2       = serial_tx
    tio3       = high_z
    tio4       = high_z
    pdid       = high_z
    pdic       = high_z
    nrst       = high
    glitch_hp  = True
    glitch_lp  = False
    extclk_src = hs1
    hs2        = clkgen
    target_pwr

In [ ]:
#This block can be used to reload the FPGA if things got really messed up
if False:
    scope.scopetype
    loader = scope.scopetype.getFWConfig().loader
    loader.driver.FPGAProgram(loader.fpga_bitstream())

In [ ]:
#You often need to re-run this
scope.glitch.resetDcms()

In [ ]:
#Reset the target device
reset_target(scope)

In [ ]:
#Read any output
target.read(100)

### ChipShover Connection

In [174]:
from chipshover import chipshover
cs = chipshover.ChipShover('com3')

In [175]:
cs.home()

b'echo:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\necho:busy: processing\nX:0.00 Y:0.00 Z:200.00 E:0.00 Count X:0 Y:0 Z:1280000\nok\n'

In [176]:
#The following is limits of the surface scanning on our setup
#Lower Limit
cs.move(x=47.6, y=8.6)
#Upper Limit
#cs.move(x=50.4, y=11.4)

#sw = cs.sweep_x_y(47.6, 50.4, 8.6, 11.4, step=0.2, z_plunge=1)

In [188]:
#This depth should be just above surface
cs.move_zdepth(36.5)

In [178]:
#This depth should be touching surface
cs.move_zdepth(37.5)

In [199]:
#Wafer tip location - found by nudging in code by 0.1mm and visually looking for corner touch
#cs.move_zdepth(37.0) # Need to be almost on surface, no pressure on it though or may break off edge of wafer
#cs.move(x=46.8, y=8.3) #Limit 1
#cs.move(x=51.1, y=12.3) #Limit 2

print("Y-size = %f "%(12.3 - 8.3))
print("X-size = %f" %(51.1-46.8))

Y-size = 4.000000 
X-size = 4.300000


## Testing FLASH Memory Read/Write

Checking for invalid flash memory. This function is used to check for corruption of flash memory.

In [ ]:
import time
scope.io.target_pwr = False
time.sleep(1)
scope.io.target_pwr = True

stm = prog()
stm.scope = scope
stm.open()
stm.find()
stm32f = stm.stm32prog()

In [106]:
#Erase memory only
stm32f.cmdEraseMemory()

#Erase & Program memory (disconnects after as well) - uncomment this
#fw_path = "simpleserial-aes-CW308_STM32F4.hex"
#cw.program_target(scope, prog, fw_path)

Extended erase (0x44), this can take ten seconds or more


In [58]:
import time
scope.io.target_pwr = False
time.sleep(1)
scope.io.target_pwr = True

stm = prog()
stm.scope = scope
stm.open()
stm.find()
stm32f = stm.stm32prog()

Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx


In [59]:
data_ref = stm32f.cmdReadMemory(0x8000000, 0x100)
print(data_ref)

[0, 0, 2, 32, 253, 15, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [60]:
stm.close()

Serial baud rate = 115200


In [80]:
def check_flash(force=False):
    import time
    scope.io.target_pwr = False
    time.sleep(1)
    scope.io.target_pwr = True

    stm = prog()
    stm.scope = scope
    stm.open()
    stm.find()
    stm32f = stm.stm32prog()
    
    data_check = stm32f.cmdReadMemory(0x8000000, 0x100)
    stm.close()
    
    if force is False:
        if data_check == data_ref:
            return False
    
    print("Status of FLASH:")
    print(data_check)
    
    fw_path = "../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
    cw.program_target(scope, prog, fw_path)
    
    test_mode = 1

    #Test 1 - 7.37 MHz device clock
    if test_mode == 1:
        scope.clock.adc_src = "clkgen_x4"
        scope.clock.clkgen_freq = 7372800
        target.baud = 38400

    #Test 2 - 40 MHz device clock
    elif test_mode == 2:
        scope.clock.adc_src = "clkgen_x1"
        scope.clock.clkgen_freq = 40E6
        target.baud = (scope.clock.clkgen_freq / 7372800) * 38400

    scope.glitch.resetDcms()
    
    return True

In [81]:
check_flash()

Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Serial baud rate = 115200
Status of FLASH:
[0, 0, 2, 32, 253, 15, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Extended erase (0x44), this can 

True

## Basic Glitch Loop

This performs a test of a base glitch loop.

In [ ]:
%%bash
cd ../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM="CW308_STM32F4" CRYPTO_TARGET="NONE" STM32F4_WLCSP="1"

In [148]:
fw_path = "../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
print(fw_path)
cw.program_target(scope, prog, fw_path)

../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-CW308_STM32F4.hex
Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5323 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5323 bytes
Serial baud rate = 115200


In [47]:
scope.io.target_pwr = True

In [163]:
test_mode = 1

#Test 1 - 7.37 MHz device clock
if test_mode == 1:
    scope.clock.adc_src = "clkgen_x4"
    scope.clock.clkgen_freq = 7372800
    target.baud = 38400

#Test 2 - 40 MHz device clock
elif test_mode == 2:
    scope.clock.adc_src = "clkgen_x1"
    scope.clock.clkgen_freq = 40E6
    target.baud = (scope.clock.clkgen_freq / 7372800) * 38400
    
scope.glitch.resetDcms()

Serial baud rate = 38400


In [164]:
reset_target(scope)
set_voltage(0)
scope.io.glitch_hp = False
scope.arm()
target.write("g\n")
ret = scope.capture()
val = target.simpleserial_read_witherrors('r', 4) #For loop check
print(val)
scope.io.glitch_hp = True

{'valid': False, 'payload': None, 'full_response': '\x08(\x08\x00 \x00\x00\x00\x00rRESET   \nrC4090000\nz00\n', 'rv': None}


#### Main XY Scan Loop

In [173]:
import chipwhisperer.common.results.glitch as glitch
from tqdm import tqdm_notebook
import struct
import numpy as np
cs.move_zdepth(36.7)

sw = cs.sweep_x_y(47.6, 50.4, 8.6, 11.4, step=0.2, z_plunge=1) #Original
#sw = cs.sweep_x_y(50.4, 50.5, 8.6, 11.4, step=0.2, z_plunge=1) #Adding another strip along z
#sw = cs.sweep_x_y(47.6, 50.5, 11.6, 11.8, step=0.2, z_plunge=1) #Adding 2 more strips along y
scope.io.glitch_lp = False

# ChipShover generates a yield function that will iterate through X/Y locations.
for x,y in sw:
    
    print("%f-%f"%(x, y))
       
    gc = glitch.GlitchController(groups=["success", "reset", "normal", "flasherr"], parameters=["voltage", "width"])
    gc.display_stats()
    
    # Init target - power cycle, do reset, flush all old garbage from serial buffers
    scope.io.target_pwr = False
    time.sleep(0.25)
    scope.io.target_pwr = True
    time.sleep(0.2)
    reset_target(scope)
    target.flush()

    # Use a fixed offset from the trigger - we are not experimenting with time location here.
    scope.glitch.ext_offset = 200

    mv_start = 500
    mv_end = 10000

    # Faster run for XY scanning
    mv_step_size = 250
    trial_per_point = 2

    repeat_step_size = 1
    repeat_start = 5
    repeat_max = 60

    mode = "counter"
    #mode = "control-flow"

    for mv_setting in tqdm_notebook(range(mv_start, mv_end, mv_step_size)):
        v_setting = mv_setting / 1000.0
        set_voltage(v_setting)
        for trial in range(0, trial_per_point):
            resets_in_a_row = 0
            for repeat in range(repeat_start, repeat_max, repeat_step_size):

                scope.glitch.repeat = repeat

                #Fast check - stop increasing beyond some point if we see a lot of resets
                if resets_in_a_row > 2:
                    #repeat_max = repeat
                    #break
                        # Init target - power cycle, do reset, flush all old garbage from serial buffers
                    scope.io.target_pwr = False
                    time.sleep(0.25)
                    scope.io.target_pwr = True
                    time.sleep(0.2)
                    reset_target(scope)
                    target.flush()
                    
                if resets_in_a_row > 5:
                    if check_flash() == False:
                        print("Flash OK...")
                    else:
                        gc.add("flasherr", (v_setting, scope.glitch.repeat))
                        
                if resets_in_a_row > 10:
                    check_flash(True)
                    break

                if scope.adc.state:
                    # can detect crash here (fast) before timing out (slow)
                    print("Trigger still high!")            

                    reset_target(scope)
                    target.flush()

                    resets_in_a_row += 1

                scope.io.glitch_lp = True
                scope.arm()

                #Do glitch loop
                target.write("g\n")
                ret = scope.capture()
                val = target.simpleserial_read_witherrors('r', 4)#For loop check
                scope.io.glitch_lp = False

                if ret:
                    print('Timeout - no trigger')
                    gc.add("reset", (v_setting, scope.glitch.repeat))

                    resets_in_a_row += 1

                    #Device is slow to boot?
                    reset_target(scope)
                    target.flush()

                else:
                    if val['valid'] is False:
                        gc.add("reset", (v_setting, scope.glitch.repeat))
                        resets_in_a_row += 1
                    else:

                        resets_in_a_row = 0

                        #print(val['payload'])

                        #Code-flow check returns success/fail
                        if mode == "control-flow":
                            gcnt = struct.unpack("<b", val['payload'])[0] #for code-flow check
                            expected_return = 0

                        elif mode == "counter":
                            #Value check - returns loop counter
                            gcnt = struct.unpack("<I", val['payload'])[0]
                            expected_return = 2500

                        else:
                            raise ValueError("Uknown mode")

                        #print(gcnt)
                        if gcnt != expected_return:
                            gc.add("success", (v_setting, scope.glitch.repeat), metadata=val['payload'])
                            print(val['payload'])
                            print("🐙", end="")
                        else:
                            gc.add("normal", (v_setting, scope.glitch.repeat))  
                            
    np.save("glitchloop_7_37mhz_location_%d_%d_500mV_10V_unit4.npy"%(round(x*1000), round(y*1000)), gc.results)

47.600000-11.600000


IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

IntText(value=0, description='flasherr count:', disabled=True)

...

Timeout - no trigger
Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Serial baud rate = 115200
Flash OK...


Timeout - no trigger
Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Serial baud rate = 115200
Flash OK...


Timeout - no trigger
Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Serial baud rate = 115200
Flash OK...


Timeout - no trigger
Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Serial baud rate = 115200
Flash OK...


Timeout - no trigger
Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Serial baud rate = 115200
Flash OK...
Serial baud rate = 115200
Detected known STMF32: STM32F40xxx/41xxx
Serial baud rate = 115200
Status of FLASH:
[0, 0, 2, 32, 253, 15, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 69, 16, 0, 8, 69, 16, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [147]:
print(x)
print(y)

47.6
10.599999999999993
